# Bird's Simple Spectral Model
Reference: R.E Bird and C. Riordan, Am. Met. Soc, 1986

In [ ]:
%matplotlib inline

In [138]:
from math import*
import csv
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt
pd.set_option('display.mpl_style', 'default') # For prettier graphs


Define the variables/functions needed for the computation of the **cos(AOI)**.

In [123]:
def day_angle():
        day_angle = 6.283185 * ((float(nday) - 1)/365)
        return (day_angle)
    
def declination_f(a):
        declination_f = (0.006918-0.399912*cos(a)+0.070257*sin(a)-0.006758*cos(2*a) +
                         0.000907*sin(2*a)-0.002697*cos(3*a)+0.00148*sin(3*a))*(180/3.14159)
        return (declination_f)

def eqntime_f(a):
        eqntime_f = (0.000075+0.001868*cos(a)-0.032077*sin(a)-0.014615*cos(2*a) -0.040849*sin(2*a))*(229.18)
        return (eqntime_f)

def hour_angle_f(a):
        hour_angle_f = 15*(float(hour)+float(minute)/60+a/60+((floor(float(lon)/15)*15-float(lon))*4)/60+12)-360
        return (hour_angle_f)

def zenith_ang_f(a,b):
        zenith_ang_f = degrees(acos(cos(radians(a))*cos(radians(float(lat)))*cos(radians(b))+
                                    sin(radians(a))*sin(radians(float(lat)))))
        return (zenith_ang_f)

def solar_azim_f(a,b,c):
        solar_azim_f = 180-degrees(asin(-cos(radians(a))*sin(radians(b))/cos(radians(90-c))))
        return (solar_azim_f)

def surf_azim_f(a):
        if float(fixed_tilt) >= 0:
                surf_azim_f = (float(azim)-180)/(180/3.14159)
        else:
                surf_azim_f = a/(180/3.14159)
        return (surf_azim_f)


### cos(AOI) compuation 

In [124]:
def cos_aoi_f(a,b,c):
        if float(fixed_tilt) >= 0:
                cos_aoi_f = sin(radians(a))*sin(radians(float(lat)))*cos(radians(float(slope)))-\
                sin(radians(a))*cos(radians(float(lat)))*sin(radians(float(slope)))*cos(b) + \
                cos(radians(a))*cos(radians(float(lat)))*cos(radians(float(slope)))*cos(radians(c)) + \
                cos(radians(a))*sin(radians(float(lat)))*sin(radians(float(slope)))*cos(b)*cos(radians(c)) + \
                cos(radians(a))*sin(radians(float(slope)))*sin(b)*sin(radians(c))
        else:
                cos_aoi_f = 1
        return (cos_aoi_f)

Define the variables/functions for the **Direct Normal Irradiance** computation

In [125]:
def rad_vec_f(a):
        rad_vec_f = 1.00011+0.034221*cos(a)+0.00128*sin(a)+0.000719*cos(2*a) +0.000077*sin(2*a)
        return (rad_vec_f)

def am_f(a):
        am_f = 1/(cos(radians(a))+0.15*pow((93.885-a),(-1.253)))
        return (am_f)

def amp_f(a):
        amp_f = a*(float(press)/1013)
        return (amp_f)

def tr_f(a):
        tr_f = exp(-a/(pow(float(wavelength),4)*(115.6406-1.335/pow(float(wavelength),2))))
        return (tr_f)

def zrad_f(a):
        zrad_f = radians(a)
        return (zrad_f)

def amoz_f(a):
        amoz_f = (1+22/6370)/pow((pow(cos(a),2)+2*(22/6370)),0.5)
        return (amoz_f)

def toz_f(a):
        toz_f = exp(-float(o3_abs)*a*float(o3))
        return (toz_f)

def tuga_f(a):
        tuga_f = exp(-1.41*float(uni_abs)*a/pow((1+118.93*float(uni_abs)*a),0.45))
        return (tuga_f)

def th20_f(a):
        th20_f = exp(-0.2385*float(water_abs)*float(precip_h2o)*a/pow((1+20.07*float(water_abs)*float(precip_h2o)*a),0.45))
        return (th20_f)

def dela_f():
        dela_f = float(aod)*(pow((float(wavelength)/0.5),(-float(alpha))))
        return (dela_f)

def ta_f(a,b):
        ta_f = exp(-b*a)
        return (ta_f)

def direct_irr_f(a,b,c,d,e,f):
        direct_irr_f = float(etr)*a*b*c*d*e*f
        return (direct_irr_f)

Defining the variables for the **Diffuse or Scattered Irradiance** computation

In [126]:
def omegl_f():
        omegl_f = float(omeg)*exp(-float(omegp)*pow(log(float(wavelength)/0.4),2))
        return (omegl_f)

def taa_f(a,b,c):
        taa_f = exp(-(1-a)*b*c)
        return (taa_f)

def dray_f(a,b,c,d,e,f,g):
        dray_f = float(etr)*cos(c)*d*e*b*g*(1-pow(a,0.95))*0.5*f
        return (dray_f)

def alg_f():
        alg_f = log(1-float(asym))
        return (alg_f)

def afs_f(a):
        afs_f = a*(1.459+a*(0.1591+a*0.4129))
        return (afs_f)

def bfs_f(a):
        bfs_f = a*(0.0783+a*(-0.3824-a*0.5874))
        return (bfs_f)

def tas_f(a,b,c):
        tas_f = exp(-b*a*c)
        return (tas_f)

def daer_f(b,c,d,e,f,g,h,i,j,k):
        daer_f = float(etr)*cos(b)*c*d*e*f*pow(g,1.5)*(1-h)*(1-0.5*exp((i+j*cos(b))*cos(b)))*k
        return (daer_f)

def twp_f():
        twp_f = exp(-0.2385*float(water_abs)*float(precip_h2o)*1.8/(pow((1+20.07*float(water_abs)*float(precip_h2o)*1.8),0.45)))
        return (twp_f)

def tup_f():
        tup_f = exp(-1.41*float(uni_abs)*1.8/(pow((1+118.93*float(uni_abs)*1.8),0.45)))
        return (tup_f)

def tasp_f(a,b):
        tasp_f = exp(-b*a*1.8)
        return (tasp_f)

def taap_f(a,b):
        taap_f = exp(-(1-b)*a*1.8)
        return (taap_f)

def fsp_f(a,b):
        fsp_f = 1-0.5*exp((a+b/1.8)/1.8)
        return (fsp_f)

def trp_f():
        trp_f = exp(-1.8/(pow(float(wavelength),4)*(115.6406-1.335/pow(float(wavelength),2))))
        return (trp_f)

def rhoa_f(a,b,c,d,e,f):
        rhoa_f = a*b*c*(0.5*(1-d)+(1-e)*d*(1-f))
        return (rhoa_f)

def drgd_f(a,b,c,d,e):
        drgd_f = (a*cos(b)+(c+d))*float(albedo)*e/(1-float(albedo)*e)
        return (drgd_f)

def hzdif_f(a,b,c):
        if float(wavelength) <= 0.45:
                hzdif_f = (a+b+c)*(pow((float(wavelength)+0.55),1.8))
        else:
                hzdif_f = a+b+c
        return (hzdif_f)

def ref_f(a,b,c):
        ref_f = (b*cos(c)+a)*float(albedo)*(1-cos(radians(float(slope))))/2
        return (ref_f)

def difsc_f(a,b,c,d):
        difsc_f = a*(b/float(etr))*c/cos(d)
        return (difsc_f)

def difsi_f(a,b):
        difsi_f = 0.5*a*(1-(b/float(etr)))*(1+cos(radians(float(slope)))) 
        return (difsi_f)

def diffuse_irr_f(a,b,c):
        diffuse_irr_f = (a+b+c)
        return (diffuse_irr_f)

### Total Irradiance Computation
**Total Irradiance** = **cos(AOI)**\* **Direct Normal Irradiance** + **Scattered Irradiance**

In [127]:
def total_irr_f(a,b,c):
        total_irr_f = a*(b)+c
        return (total_irr_f)

Load user_input.csv

In [128]:
user_input = pd.read_csv("I:\Proj 4\Jerine\codes\user_input.csv")
user_input_i = user_input.set_index("User input")
print user_input_i

            04/01/2016 8:30  04/01/2016 8:35  04/01/2016 8:40  \
User input                                                      
fixed_tilt         0.000000         0.000000         0.000000   
hour               8.000000         8.000000         8.000000   
minute            30.000000        35.000000        40.000000   
azim               0.000000         0.000000         0.000000   
aod                0.370000         0.370000         0.370000   
alpha              1.800000         1.800000         1.800000   
albedo             0.115600         0.115600         0.115600   
o3                 0.434600         0.434600         0.434600   
precip_h2o         2.982000         2.982000         2.982000   
slope              0.000000         0.000000         0.000000   
press            989.600370       989.600370       989.600370   
nday              92.000000        92.000000        92.000000   
omeg               0.945000         0.945000         0.945000   
omegp              0.0950

In [129]:
wp_columns = len(user_input.columns)
wp_rows= len(user_input.index)
print "User Input No. of Columns:", wp_columns
print "User Input No. of Rows:",wp_rows

User Input No. of Columns: 115
User Input No. of Rows: 17


Loop for waypoint columns

In [130]:
for wp_column in range(1, int(wp_columns-1)):
        fixed_tilt = user_input_i.loc["fixed_tilt",:][wp_column]
        hour = user_input_i.loc["hour",:][wp_column]
        minute = user_input_i.loc["minute",:][wp_column]
        azim = user_input_i.loc["azim",:][wp_column]
        aod = user_input_i.loc["aod",:][wp_column]
        alpha = user_input_i.loc["alpha",:][wp_column]
        albedo = user_input_i.loc["albedo",:][wp_column]
        o3 = user_input_i.loc["o3",:][wp_column]
        precip_h2o = user_input_i.loc["precip_h2o",:][wp_column]
        slope = user_input_i.loc["slope",:][wp_column]
        press = user_input_i.loc["press",:][wp_column]
        nday = user_input_i.loc["nday",:][wp_column]
        omeg = user_input_i.loc["omeg",:][wp_column]
        omegp = user_input_i.loc["omegp",:][wp_column]
        lat = user_input_i.loc["lat",:][wp_column]
        lon = user_input_i.loc["long",:][wp_column]
        asym = user_input_i.loc["asym",:][wp_column]

In [131]:
fixed_model_data = pd.read_csv("I:\Proj 4\Jerine\codes\model_data.csv")
#fixed_model_data_i = fixed_model_data.set_index("um")
print fixed_model_data

     wavelength     ETR   Water Abs  O3 Abs    Uni Abs
0         0.300   535.9      0.0000  10.000    0.00000
1         0.305   558.3      0.0000   4.800    0.00000
2         0.310   622.0      0.0000   2.700    0.00000
3         0.315   692.7      0.0000   1.350    0.00000
4         0.320   715.1      0.0000   0.800    0.00000
5         0.325   832.9      0.0000   0.380    0.00000
6         0.330   961.9      0.0000   0.160    0.00000
7         0.335   931.9      0.0000   0.075    0.00000
8         0.340   900.6      0.0000   0.040    0.00000
9         0.345   911.3      0.0000   0.019    0.00000
10        0.350   975.5      0.0000   0.007    0.00000
11        0.360   975.9      0.0000   0.000    0.00000
12        0.370  1119.9      0.0000   0.000    0.00000
13        0.380  1103.8      0.0000   0.000    0.00000
14        0.390  1033.8      0.0000   0.000    0.00000
15        0.400  1479.1      0.0000   0.000    0.00000
16        0.410  1701.3      0.0000   0.000    0.00000
17        

In [132]:
fmd_columns = len(fixed_model_data.columns)
fmd_rows= len(fixed_model_data.index)
print "Fixed Model Data No. of Columns:", fmd_columns
print "Fixed Model Data No. of Rows:",fmd_rows

Fixed Model Data No. of Columns: 5
Fixed Model Data No. of Rows: 122


Loop for fixed model data.

In [137]:
        for fmd_row in range(1,int(fmd_rows-1)):
                wavelength = fixed_model_data.loc[:, "wavelength"][fmd_row]
                etr = fixed_model_data.loc[:, "ETR"][fmd_row]
                water_abs = fixed_model_data.loc[:, "Water Abs"][fmd_row]
                o3_abs = fixed_model_data.loc[:, "O3 Abs"][fmd_row]
                uni_abs = fixed_model_data.loc[:, "Uni Abs"][fmd_row]
                
                day_angl = day_angle()
                
                declination = declination_f(day_angl)
               
                eqntime = eqntime_f(day_angl)
             
                hour_angle = hour_angle_f(eqntime)
             
                zenith_ang = zenith_ang_f(declination, hour_angle)
            
                solar_azim = solar_azim_f(declination, hour_angle, zenith_ang)
            
                surf_azim = surf_azim_f(solar_azim)
          
                cos_aoi = cos_aoi_f(declination, surf_azim, hour_angle)
           
                rad_vec = rad_vec_f(day_angl)
             
                
                am = am_f(zenith_ang)
                amp = amp_f(am)
                tr = tr_f(amp)
                zrad = zrad_f(zenith_ang)
                amoz = amoz_f(zrad)
                toz = toz_f(amoz)
                tuga = tuga_f(amp)
                th20 = th20_f(am)
                dela = dela_f()
                ta = ta_f(am, dela)
                direct_irr = direct_irr_f(rad_vec, tr, toz, tuga, th20, ta)
                omegl = omegl_f()
                taa = taa_f(omegl, dela, am)
                dray = dray_f(tr, tuga, zrad, toz, th20, rad_vec, taa)
                alg = alg_f()
                afs = afs_f(alg)
                bfs = bfs_f(alg)
                tas = tas_f(dela, omegl, am)
                daer = daer_f(zrad, toz, th20, tuga, taa, tr, tas, afs, bfs, rad_vec)
                twp = twp_f()
                tup = tup_f()
                tasp = tasp_f(dela, omegl)
                taap = taap_f(dela, omegl)
                fsp = fsp_f(afs, bfs)
                trp = trp_f()
                rhoa = rhoa_f(tup, twp, taap, trp, fsp, tasp)
                drgd = drgd_f(direct_irr, zrad, dray, daer, rhoa)
                hzdif = hzdif_f(dray, daer, drgd)
                ref = ref_f(hzdif, direct_irr, zrad)
                difsc = difsc_f(hzdif, direct_irr, cos_aoi, zrad)
                difsi = difsi_f(hzdif, direct_irr)
                diffuse_irr = diffuse_irr_f(ref, difsc, difsi)
                total_irradiance = total_irr_f(direct_irr, cos_aoi, diffuse_irr)
                
                print (wavelength, total_irradiance)
                
#end inside loop for data per WP column
        print ("\n")
#end loop for WP columns

(0.30499999999999999, 8.097429346746889e-13)
(0.31, 4.485902600941331e-07)
(0.315, 0.002320984389647879)
(0.32000000000000001, 0.07618658485522833)
(0.32500000000000001, 1.2534246069464352)
(0.33000000000000002, 5.87431506515738)
(0.33500000000000002, 9.947418360621278)
(0.34000000000000002, 12.28944342476256)
(0.34499999999999997, 14.550497494889216)
(0.34999999999999998, 17.20970660283333)
(0.35999999999999999, 18.840982679502776)
(0.37, 22.55230974551551)
(0.38, 23.089818587752774)
(0.39000000000000001, 22.375384146333136)
(0.40000000000000002, 32.998783693809514)
(0.40999999999999998, 38.985607031100436)
(0.41999999999999998, 40.82966378043626)
(0.42999999999999999, 38.010181420687104)
(0.44, 44.79673181809657)
(0.45000000000000001, 48.76917050941821)
(0.46000000000000002, 48.61688929558787)
(0.46999999999999997, 46.22500981128162)
(0.47999999999999998, 45.51677029181906)
(0.48999999999999999, 40.59461603823106)
(0.5, 38.51158690247633)
(0.51000000000000001, 36.43414230154882)
(0.5

### Visualization

In [141]:
x = wavelength
y = total_irradiance

ValueError: If using all scalar values, you must pass an index